1. License Plate Detection: The first step is to detect the License plate from the car. We will use the contour option in OpenCV to detect for rectangular objects to find the number plate. The accuracy can be improved if we know the exact size, color and approximate location of the number plate.

2. Character Segmentation: Once we have detected the License Plate we have to crop it out and save it as a new image. Again this can be done easily using OpenCV.

3. Character Recognition: Now, the new image that we obtained in the previous step is sure to have some characters (Numbers/Alphabets) written on it. So, we can perform OCR (Optical Character Recognition) on it to detect the number

In [2]:
import cv2
import imutils
import numpy as np
import pytesseract

### Resize the image to the required size and then grayscale it.Normally using a bilateral filter (Blurring) will remove the unwanted details from an image.Syntax--> cv2.bilateralFilter(source_image, diameter of pixel, sigmaColor, sigmaSpace). 
You can increase the sigma color and sigma space from 15 to higher values to blur out more background information, but be careful that the useful part does not get blurred. The output image is shown below, as you can see the background details (tree and building) are blurred in this image. This way we can avoid the program from concentrating on these regions later.

In [10]:
img = cv2.imread('dataset/3.jpg',cv2.IMREAD_COLOR)
img = cv2.resize(img, (600,400) )

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
gray = cv2.bilateralFilter(gray, 13, 15, 15) 
cv2.imshow('gray',gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

EDGE DETECTION--The syntax--> destination_image = cv2.Canny(source_image, thresholdValue 1, thresholdValue 2). The Threshold Vale 1 and Threshold Value 2 are the minimum and maximum threshold values. Only the edges that have an intensity gradient more than the minimum threshold value and less than the maximum threshold value will be displayed. 

In [22]:
edged = cv2.Canny(gray, 30, 200) 
cv2.imshow('edged',edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

Looking for contours on our image:Once the counters have been detected we sort them from big to small and consider only the first 10 results ignoring the others. In our image the counter could be anything that has a closed surface but of all the obtained results the license plate number will also be there since it is also a closed surface.
To filter the license plate image among the obtained results, we will loop though all the results and check which has a rectangle shape contour with four sides and closed figure. Since a license plate would definitely be a rectangle four sided figure.Once we have found the right counter we save it in a variable called screenCnt and then draw a rectangle box around it to make sure we have detected the license plate correctly. cv2. CHAIN_APPROX_SIMPLE returns only the endpoints that are necessary for drawing the contour line.So each contour has its own information regarding what hierarchy it is, who is its child, who is its parent etc. OpenCV represents it as an array of four values : [Next, Previous, First_Child, Parent]
It retrieves all the contours and creates a full family hierarchy list. It even tells, who is the grandpa, father, son, grandson and even beyond.

cv.findContours() function takes three arguments-

The first argument is the source image. Here we make a copy of the edged image as this function repeatedly finds contours in the image, meaning it makes the image unusable for future use. Also, the edged image makes identifying similar intensity curves easier.
The second argument is the contour retrieval mode. Here the RETR_LIST type is used to retrieve all the detected contours in the image.
The third parameter is the contour approximation method. CHAIN_APPROX_SIMPLE stores the end-points of the detected contours.


cv2.drawContours() function takes five arguments-

The first argument is the image in which the detected contours will be drawn.
The second argument is the variable that stores all the detected contours.
Then, the third argument is contour indexes. Here we use the value -1 which will use the indexes of all the contours detected in our image. Hence all of the contours will be drawn on the image.
After that, the fourth argument is the color in which contours will be drawn.
The fifth argument is the thickness of the contour curve to be drawn.
Using peri function, we calculate perimeter of each contour.


approxPolyDP 
The function cv::approxPolyDP approximates a curve or a polygon with another curve/polygon with less vertices so that the distance between them is less or equal to the specified precision. It uses the Douglas-Peucker algorithm http://en.wikipedia.org/wiki/Ramer-Douglas-Peucker_algorithm

Parameters
curve	Input vector of a 2D point stored in std::vector or Mat
approxCurve	Result of the approximation. The type should match the type of the input curve.
epsilon	Parameter specifying the approximation accuracy. This is the maximum distance between the original curve and its approximation.
closed	If true, the approximated curve is closed (its first and last vertices are connected). Otherwise, it is not closed.


arclength
Calculates a contour perimeter or a curve length.

The function computes a curve length or a closed contour perimeter.

Parameters
curve	Input vector of 2D points, stored in std::vector or Mat.
closed	Flag indicating whether the curve is closed or not.

In [23]:
contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None

for c in contours:
    
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
 
    if len(approx) == 4:
        screenCnt = approx
        break

if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
     detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

    
cv2.imshow('IMG',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Masking the entire picture except for the place where the number plate is.

In [24]:
mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv2.bitwise_and(img,img,mask=mask)
cv2.imshow('IMG',new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

The next step in Number Plate Recognition is to segment the license plate out of the image by cropping it and saving it as a new image. We can then use this image to detect the character in it.

In [25]:
(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]
cv2.imshow('IMG',Cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

The Final step in this Number Plate Recognition is to actually read the number plate information from the segmented image. We will use the pytesseract package to read characters from image

In [26]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

text = pytesseract.image_to_string(Cropped, config='--psm 11')
print("Detected license plate Number is:",text[1:])

Detected license plate Number is: HR26DK8337



In [27]:
img = cv2.resize(img,(500,300))
Cropped = cv2.resize(Cropped,(400,200))
cv2.imshow('car',img)
cv2.imshow('Cropped',Cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()